In [ ]:
# run # In the folder where ivr_simulator.html is saved:
#python3 -m http.server 8000
# Then vist the below code file saved as html - http://localhost:8000/ivr_simulator.html

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hybrid IVR Simulation</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
        body {
            font-family: 'Inter', sans-serif;
            background-color: #f3f4f6;
        }
        .chat-container {
            max-height: 80vh;
            overflow-y: auto;
            border-radius: 0.75rem;
        }
        .chat-message {
            max-width: 70%;
            padding: 0.75rem 1.25rem;
            border-radius: 1.5rem;
            margin-bottom: 0.75rem;
        }
    </style>
</head>
<body class="flex items-center justify-center min-h-screen p-4">
    <div class="bg-white shadow-xl rounded-2xl w-full max-w-xl flex flex-col h-[90vh]">
        <div class="bg-blue-600 text-white p-4 rounded-t-2xl shadow-md text-center">
            <h1 class="text-xl font-bold">Automated Banking IVR</h1>
            <p class="text-sm opacity-90">How may I help you today?</p>
        </div>
        <div id="chat-box" class="chat-container flex-grow p-4 space-y-4">
            <!-- Initial welcome message from the IVR -->
            <div class="flex justify-start">
                <div class="chat-message bg-gray-200 text-gray-800">
                    Welcome to the Gemini Bank IVR. I can help with balance inquiries, recent transactions, or a transfer. What would you like to do?
                </div>
            </div>
            <!-- Loading indicator -->
            <div id="loading-indicator" class="flex justify-start hidden">
                <div class="chat-message bg-gray-200 text-gray-800 animate-pulse">
                    Thinking...
                </div>
            </div>
        </div>
        <div class="p-4 border-t border-gray-200 flex rounded-b-2xl">
            <input type="text" id="user-input" placeholder="Type your message..." class="flex-grow p-3 border border-gray-300 rounded-full focus:outline-none focus:ring-2 focus:ring-blue-500 transition-all">
            <button id="send-button" class="ml-2 bg-blue-600 text-white p-3 rounded-full hover:bg-blue-700 transition-colors shadow-md focus:outline-none focus:ring-2 focus:ring-blue-500">
                <svg xmlns="http://www.w3.org/2000/svg" class="h-6 w-6" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                    <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M14 5l7 7m0 0l-7 7m7-7H3" />
                </svg>
            </button>
        </div>
    </div>

    <script>
        document.addEventListener('DOMContentLoaded', () => {
            const chatBox = document.getElementById('chat-box');
            const userInput = document.getElementById('user-input');
            const sendButton = document.getElementById('send-button');
            const loadingIndicator = document.getElementById('loading-indicator');

            // Function to add a message to the chat box
            function addMessage(text, sender) {
                const messageContainer = document.createElement('div');
                const messageBubble = document.createElement('div');
                messageBubble.textContent = text;
                messageBubble.className = 'chat-message';

                if (sender === 'user') {
                    messageContainer.className = 'flex justify-end';
                    messageBubble.classList.add('bg-blue-500', 'text-white');
                } else {
                    messageContainer.className = 'flex justify-start';
                    messageBubble.classList.add('bg-gray-200', 'text-gray-800');
                }

                messageContainer.appendChild(messageBubble);
                chatBox.appendChild(messageContainer);
                chatBox.scrollTop = chatBox.scrollHeight;
            }

            // A function to simulate intent recognition (like Amazon Lex)
            function getLexIntent(query) {
                const lowerQuery = query.toLowerCase();
                if (lowerQuery.includes('balance')) {
                    return { intent: 'GetAccountBalance', response: "Sure, your current balance is $1,250.75." };
                } else if (lowerQuery.includes('lost card') || lowerQuery.includes('stolen card')) {
                    return { intent: 'ReportLostCard', response: "I'm sorry to hear that. I have initiated the process to freeze your card. A new one will be sent in 3-5 business days." };
                } else if (lowerQuery.includes('transfer')) {
                    return { intent: 'InitiateTransfer', response: "What is the amount and the recipient of the transfer?" };
                }
                return null;
            }

            // Function to handle the user's query
            async function handleQuery() {
                const query = userInput.value.trim();
                if (query === '') return;

                addMessage(query, 'user');
                userInput.value = '';

                // Show the loading indicator while processing
                loadingIndicator.classList.remove('hidden');

                // Step 1: Simulate Amazon Lex for high-priority intents
                const lexResponse = getLexIntent(query);
                if (lexResponse) {
                    loadingIndicator.classList.add('hidden');
                    addMessage(lexResponse.response, 'ivr');
                    return;
                }

                // Step 2: If no specific intent is found, use a generative model (Hugging Face / SageMaker)
                // We'll simulate this with the Gemini API to provide a dynamic, generative response.
                try {
                    const prompt = `You are a helpful and polite bank customer service assistant. Respond to the following customer query: "${query}"`;
                    let chatHistory = [];
                    chatHistory.push({ role: "user", parts: [{ text: prompt }] });
                    const payload = { contents: chatHistory };
                    const apiKey = "";
                    const apiUrl = `https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-05-20:generateContent?key=${apiKey}`;
                    
                    const response = await fetch(apiUrl, {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify(payload)
                    });

                    if (!response.ok) {
                        throw new Error(`API call failed with status: ${response.status}`);
                    }

                    const result = await response.json();
                    
                    if (result.candidates && result.candidates.length > 0 &&
                        result.candidates[0].content && result.candidates[0].content.parts &&
                        result.candidates[0].content.parts.length > 0) {
                        const generatedText = result.candidates[0].content.parts[0].text;
                        addMessage(generatedText, 'ivr');
                    } else {
                        addMessage("I'm sorry, I couldn't understand that. Can you please rephrase?", 'ivr');
                    }
                } catch (error) {
                    console.error("Error during generative API call:", error);
                    addMessage("I'm sorry, there was a problem with my service. Please try again later.", 'ivr');
                } finally {
                    // Hide the loading indicator
                    loadingIndicator.classList.add('hidden');
                }
            }

            // Event listeners for sending messages
            sendButton.addEventListener('click', handleQuery);
            userInput.addEventListener('keypress', (event) => {
                if (event.key === 'Enter') {
                    handleQuery();
                }
            });
        });
    </script>
</body>
</html>
